In [1]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [5]:
# load data
dataset_train = numpy.genfromtxt("fashionmnist/fashion-mnist_train.csv", delimiter=",", dtype=None, names=True)
dataset_test = numpy.genfromtxt("fashionmnist/fashion-mnist_test.csv", delimiter=",", dtype=None, names=True)

In [12]:
print(dataset_train.shape)
print(dataset_test.shape)

(60000,)
(10000,)


In [16]:
import pandas as pd
train_df = pd.read_csv("fashionmnist/fashion-mnist_train.csv")
test_df = pd.read_csv("fashionmnist/fashion-mnist_test.csv")

In [17]:
train_df.shape

(60000, 785)

In [19]:
train_df.shape[1]

785

In [24]:
X = train_df.iloc[:, 1:785].values
y = train_df.iloc[:, 0].values

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=seed)

In [ ]:
# plot 4 images as gray scale
plt.subplot(221)
plt.imshow(X_train[0], cmap=plt.get_cmap('gray')) 
plt.subplot(222)
plt.imshow(X_train[1], cmap=plt.get_cmap('gray')) 
plt.subplot(223)
plt.imshow(X_train[2], cmap=plt.get_cmap('gray')) 
plt.subplot(224)
plt.imshow(X_train[3], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

In [30]:
# reshape to be [samples][channels][width][height]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') 
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

In [27]:
# normalize inputs from 0-255 to 0-1
X_train = X_train/255
X_test = X_test/255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [31]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(28, 28, 1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

In [32]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="valid", activation="relu", input_shape=(28, 28, 1...)`
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 42000 samples, validate on 18000 samples
Epoch 1/10
 - 37s - loss: 0.5567 - acc: 0.8063 - val_loss: 0.3853 - val_acc: 0.8648
Epoch 2/10
 - 40s - loss: 0.3742 - acc: 0.8679 - val_loss: 0.3417 - val_acc: 0.8805
Epoch 3/10
 - 38s - loss: 0.3246 - acc: 0.8854 - val_loss: 0.3046 - val_acc: 0.8934
Epoch 4/10
 - 36s - loss: 0.3015 - acc: 0.8934 - val_loss: 0.2969 - val_acc: 0.8926
Epoch 5/10
 - 41s - loss: 0.2774 - acc: 0.9015 - val_loss: 0.2751 - val_acc: 0.9040
Epoch 6/10
 - 42s - loss: 0.2605 - acc: 0.9054 - val_loss: 0.2688 - val_acc: 0.9020
Epoch 7/10
 - 37s - loss: 0.2486 - acc: 0.9095 - val_loss: 0.2748 - val_acc: 0.9014
Epoch 8/10
 - 38s - loss: 0.2338 - acc: 0.9144 - val_loss: 0.2634 - val_acc: 0.9061
Epoch 9/10
 - 49s - loss: 0.2199 - acc: 0.9194 - val_loss: 0.2493 - val_acc: 0.9125
Epoch 10/10
 - 36s - loss: 0.2064 - acc: 0.9250 - val_loss: 0.2497 - val_acc: 0.9129
CNN Error: 8.71%


In [33]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(30, 5, 5, input_shape=(28, 28, 1), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, 3, 3, activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

# build the model
model_larger = larger_model()
# Fit the model
model_larger.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model_larger.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, (5, 5), activation="relu", input_shape=(28, 28, 1...)`
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, (3, 3), activation="relu")`
  


Train on 42000 samples, validate on 18000 samples
Epoch 1/10
 - 35s - loss: 0.8019 - acc: 0.7038 - val_loss: 0.5099 - val_acc: 0.8162
Epoch 2/10
 - 37s - loss: 0.5008 - acc: 0.8154 - val_loss: 0.4385 - val_acc: 0.8433
Epoch 3/10
 - 36s - loss: 0.4386 - acc: 0.8419 - val_loss: 0.3794 - val_acc: 0.8645
Epoch 4/10
 - 38s - loss: 0.4048 - acc: 0.8538 - val_loss: 0.3651 - val_acc: 0.8693
Epoch 5/10
 - 36s - loss: 0.3786 - acc: 0.8631 - val_loss: 0.3324 - val_acc: 0.8822
Epoch 6/10
 - 37s - loss: 0.3566 - acc: 0.8702 - val_loss: 0.3348 - val_acc: 0.8798
Epoch 7/10
 - 36s - loss: 0.3437 - acc: 0.8762 - val_loss: 0.3211 - val_acc: 0.8859
Epoch 8/10
 - 37s - loss: 0.3301 - acc: 0.8801 - val_loss: 0.2984 - val_acc: 0.8932
Epoch 9/10
 - 35s - loss: 0.3203 - acc: 0.8833 - val_loss: 0.2962 - val_acc: 0.8951
Epoch 10/10
 - 41s - loss: 0.3091 - acc: 0.8852 - val_loss: 0.2853 - val_acc: 0.8983
Large CNN Error: 10.17%
